In [1]:
%%time
import pandas as pd
# texts = clean_movie_data(data)
df = pd.read_csv("data/All-seasons.csv")

CPU times: user 304 ms, sys: 384 ms, total: 688 ms
Wall time: 307 ms


In [2]:
df.head()

,Season,Episode,Character,Line
0,10,1,Stan,"You guys, you guys! Chef is going away. \n"
1,10,1,Kyle,Going away? For how long?\n
2,10,1,Stan,Forever.\n
3,10,1,Chef,I'm sorry boys.\n
4,10,1,Stan,"Chef said he's been bored, so he joining a gro..."


In [3]:
texts = df["Line"].values

In [4]:
texts[:10]

array(['You guys, you guys! Chef is going away. \n',
       'Going away? For how long?\n', 'Forever.\n', "I'm sorry boys.\n",
       "Chef said he's been bored, so he joining a group called the Super Adventure Club. \n",
       'Wow!\n',
       'Chef?? What kind of questions do you think adventuring around the world is gonna answer?!\n',
       "What's the meaning of life? Why are we here?\n",
       "I hope you're making the right choice.\n",
       "I'm gonna miss him.  I'm gonna miss Chef and I...and I don't know how to tell him! \n"], dtype=object)

In [5]:
%%time
from hedgeable_ai.functions.preprocessing.word2index import Word2IndexProcessor

processor = Word2IndexProcessor(texts[:], is_processed=False)

Using TensorFlow backend.


CPU times: user 44min 55s, sys: 16.1 s, total: 45min 11s
Wall time: 45min 12s


In [6]:
import tensorflow as tf

In [ ]:
from __future__ import print_function
import tensorflow as tf

def layer_normalization(inputs, epsilon=1e-8, scope="layer_normalization", reuse=None):
    '''Applies layer normalization.
    
    Args:
      inputs: A tensor with 2 or more dimensions, where the first dimension has
        `batch_size`.
      epsilon: A floating number. A very small number for preventing ZeroDivision Error.
      scope: Optional scope for `variable_scope`.
      reuse: Boolean, whether to reuse the weights of a previous layer
        by the same name.
      
    Returns:
      A tensor with the same shape and data dtype as `inputs`.
    '''
    with tf.variable_scope(scope, reuse=reuse):
        shape = inputs.get_shape().as_list()
        hidden_dim = shape[-1]
        mean, variance = tf.nn.moments(inputs, [-1], keep_dims=True)
        gamma = tf.get_variable("gamma", hidden_dim, initializer=tf.ones_initializer(tf.float32))
        beta = tf.get_variable("beta", hidden_dim, initializer=tf.zeros_initializer(tf.float32))
        normalized_inputs = (inputs - mean) / tf.sqrt(variance + epsilon)
        outputs = gamma * normalized_inputs + beta
    return outputs

def embedding(inputs,vocab_size, num_units, 
              zero_pad=True, scale=True,
              scope="embedding", reuse=None):
    '''Embeds a given tensor.
    Args:
      inputs: A `Tensor` with type `int32` or `int64` containing the ids
         to be looked up in `lookup table`.
      vocab_size: An int. Vocabulary size.
      num_units: An int. Number of embedding hidden units.
      zero_pad: A boolean. If True, all the values of the fist row (id 0)
        should be constant zeros.
      scale: A boolean. If True. the outputs is multiplied by sqrt num_units.
      scope: Optional scope for `variable_scope`.
      reuse: Boolean, whether to reuse the weights of a previous layer
        by the same name.
    Returns:
      A `Tensor` with one more rank than inputs's. The last dimensionality
        should be `num_units`.
        
    For example,
    
    ```
    import tensorflow as tf
    
    inputs = tf.to_int32(tf.reshape(tf.range(2*3), (2, 3)))
    outputs = embedding(inputs, 6, 2, zero_pad=True)
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        print sess.run(outputs)
    >>
    [[[ 0.          0.        ]
      [ 0.09754146  0.67385566]
      [ 0.37864095 -0.35689294]]
     [[-1.01329422 -1.09939694]
      [ 0.7521342   0.38203377]
      [-0.04973143 -0.06210355]]]
    ```
    
    ```
    import tensorflow as tf
    
    inputs = tf.to_int32(tf.reshape(tf.range(2*3), (2, 3)))
    outputs = embedding(inputs, 6, 2, zero_pad=False)
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        print sess.run(outputs)
    >>
    [[[-0.19172323 -0.39159766]
      [-0.43212751 -0.66207761]
      [ 1.03452027 -0.26704335]]
     [[-0.11634696 -0.35983452]
      [ 0.50208133  0.53509563]
      [ 1.22204471 -0.96587461]]]    
    ```    
    '''
    with tf.variable_scope(scope, reuse=reuse):
        lookup_table = tf.get_variable('lookup_table',
                                       dtype=tf.float32,
                                       shape=[vocab_size, num_units],
                                       initializer=tf.contrib.layers.xavier_initializer())
        if zero_pad:
            lookup_table = tf.concat((tf.zeros(shape=[1, num_units]),
                                      lookup_table[1:, :]), 0)
        outputs = tf.nn.embedding_lookup(lookup_table, inputs)
        
        if scale:
            outputs = outputs * tf.sqrt(num_units) 
            
    return outputs
    
def multihead_attention(queries, 
                        keys,
                        values,
                        num_units=None, 
                        num_heads=8, 
                        drop_rate=0,
                        training=True,
                        causality=False,
                        scope="multihead_attention", 
                        reuse=None):
    '''Applies multihead attention.
    
    Args:
      queries: A 3d tensor with shape of [N, T_q, C_q].
      keys: A 3d tensor with shape of [N, T_k, C_k].
      values: A 3d tensor with shape of [N, T_k, C_k].
      num_units: A scalar. Attention size.
      drop_rate: A floating point number.
      training: Boolean. Controller of mechanism for dropout.
      causality: Boolean. If true, units that reference the future are masked. 
      num_heads: An int. Number of heads.
      scope: Optional scope for `variable_scope`.
      reuse: Boolean, whether to reuse the weights of a previous layer
        by the same name.
        
    Returns
      A 3d tensor with shape of (N, T_q, C)  
    '''
    with tf.variable_scope(scope, reuse=reuse):
        # Set the fall back option for num_units
        if num_units is None:
            num_units = queries.get_shape().as_list()[-1]
        
        # Linear projections
        Q = tf.layers.dense(queries, num_units, activation=None) # (N, T_q, C)
        K = tf.layers.dense(keys, num_units, activation=None) # (N, T_k, C)
        V = tf.layers.dense(values, num_units, activation=None) # (N, T_k, C)
        
        # Split and concat
        # The size will be [N * num_heads, T_k, C_k/num_heads]
        Q_ = tf.concat(tf.split(Q, num_heads, axis=2), axis=0) # (h*N, T_q, C/h) 
        K_ = tf.concat(tf.split(K, num_heads, axis=2), axis=0) # (h*N, T_k, C/h) 
        V_ = tf.concat(tf.split(V, num_heads, axis=2), axis=0) # (h*N, T_k, C/h) 

        # Multiplication
        alignments = tf.matmul(Q_, tf.transpose(K_, [0, 2, 1])) # (h*N, T_q, T_k)
        
        # Scale
        d = K_.get_shape().as_list()[-1]
        alignments = tf.softmax(alignments / tf.sqrt(d))
  
        # Causality = Future blinding
        if causality:
            shape = tf.shape(outputs)
            masks = tf.ones(shape[1:]) # (T_q, T_k)
            masks = tf.contrib.linalg.LinearOperatorTriL(masks).to_dense() # (T_q, T_k)
            masks = tf.tile(tf.expand_dims(masks, 0), [shape[0], 1, 1]) # (h*N, T_q, T_k)
            alignments = alignments * masks
            sum_alignments = tf.reduce_sum(alignments, -1, keep_dims=True)
            sum_alignments = tf.tile(sum_alignments, [1, 1, shape[-1]])
            alingments = alignments / sum_alignments
          
        # Dropouts
        alignments = tf.layers.dropout(alignments, rate=drop_rate, training=training)
               
        # Weighted sum
        outputs = tf.matmul(alignments, V_) # ( h*N, T_q, C/h)
        
        # Restore shape
        outputs = tf.concat(tf.split(outputs, num_heads, axis=0), axis=2) # (N, T_q, C)
              
        # Residual connection
        outputs += queries
              
        # Normalize
        outputs = layer_normalization(outputs) # (N, T_q, C)
 
    return outputs

def feedforward(inputs, 
                num_units=[2048, 512],
                scope="multihead_attention", 
                reuse=None):
    '''Point-wise feed forward net.
    
    Args:
      inputs: A 3d tensor with shape of [N, T, C].
      num_units: A list of two integers.
      scope: Optional scope for `variable_scope`.
      reuse: Boolean, whether to reuse the weights of a previous layer
        by the same name.
        
    Returns:
      A 3d tensor with the same shape and dtype as inputs
    '''
    with tf.variable_scope(scope, reuse=reuse):
        # Inner layer
        params = {"inputs": inputs, "filters": num_units[0], "kernel_size": 1,
                  "activation": tf.nn.relu, "use_bias": True}
        outputs = tf.layers.conv1d(**params)
        
        # Readout layer
        params = {"inputs": outputs, "filters": num_units[1], "kernel_size": 1,
                  "activation": None, "use_bias": True}
        outputs = tf.layers.conv1d(**params)
        
        # Residual connection
        outputs += inputs
        
        # Normalize
        outputs = layer_normalization(outputs)
    
    return outputs

def label_smoothing(inputs, epsilon=0.1):
    '''Applies label smoothing. See https://arxiv.org/abs/1512.00567.
    
    Args:
      inputs: A 3d tensor with shape of [N, T, V], where V is the number of vocabulary.
      epsilon: Smoothing rate.
    
    For example,
    
    ```
    import tensorflow as tf
    inputs = tf.convert_to_tensor([[[0, 0, 1], 
       [0, 1, 0],
       [1, 0, 0]],
      [[1, 0, 0],
       [1, 0, 0],
       [0, 1, 0]]], tf.float32)
       
    outputs = label_smoothing(inputs)
    
    with tf.Session() as sess:
        print(sess.run([outputs]))
    
    >>
    [array([[[ 0.03333334,  0.03333334,  0.93333334],
        [ 0.03333334,  0.93333334,  0.03333334],
        [ 0.93333334,  0.03333334,  0.03333334]],
       [[ 0.93333334,  0.03333334,  0.03333334],
        [ 0.93333334,  0.03333334,  0.03333334],
        [ 0.03333334,  0.93333334,  0.03333334]]], dtype=float32)]   
    ```    
    '''
    K = inputs.get_shape().as_list()[-1] # number of channels
    return ((1-epsilon) * inputs) + (epsilon / K)

In [ ]:
import tensorflow as tf
from tensorflow.contrib.layers import fully_connected, conv2d
from tensorflow.contrib.layers import conv2d_transpose, flatten

from hedgeable_ai.models.nn import BaseModel
        
class FeedForward(BaseModel):
    def __init__(self, model_params, scope_name, *args, **kwargs):
        super().__init__(model_params, scope_name, *args, **kwargs)
    
    def __call__(self, x, training=True):
        with tf.variable_scope(self.scope_name, reuse=self.reuse):
            for i, params in enumerate(self.model_params):
                with tf.variable_scope('layer_' + str(i)):
                    if "is_flatten" in params and params["is_flatten"]:
                        x = flatten(x)
                    if "drop_rate" in params:
                        x = tf.layers.dropout(x, rate=params["drop_rate"], training=training)
                    # demtermine which layer to use
                    if params["name"] == "dense":
                        x = fully_connected(x, params["num_hidden"], activation_fn=None)
                    elif params["name"] == "conv2d":
                        x =  conv2d(x, params["num_filter"], params["kernel_size"],
                                          params["stride"], params["padding"], activation_fn=None)
                    elif params["name"] == "deconv2d":
                        x =  conv2d_transpose(x, params["num_filter"], params["kernel_size"],
                                          params["stride"], params["padding"], activation_fn=None)
                    elif params["name"] == "reshape":
                        x = tf.reshape(x, (-1,) + params["reshape_size"])
                    elif params["name"] == "pooling":
                        del params["name"]
                        x = tf.nn.pool(x, **params)
                    else:
                        raise NotImplementedError("No implementation for 'name'={}".format(params["name"]))         
                    if "is_batch" in params and params["is_batch"]:
                        x = tf.layers.batch_normalization(x, training=training, momentum=0.9)
                    if "activation" in params:
                        x = params["activation"](x)
            if self.reuse is False:
                self.global_scope_name = tf.get_variable_scope().name
                self.variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope=self.global_scope_name)
        self.reuse = True
        return x

In [96]:
from hedgeable_ai.models.nn.ff.core import FeedForward

class BaseAttention(object):
    def __init__(self, conf=None):
        if conf is None:
            conf = {}
        for key in self.default_conf.keys():
            if key not in conf.keys():
                conf[key] = self.default_conf[key]
        self.conf = conf
        
    def __call__(self, query, source_outputs, source_inputs, training=True):
        attentions, alignments = self._get_attentions(query, source_outputs, source_inputs, training)
        return attentions, alignments
    
    def _get_attentions(self, query, source_outputs, source_inputs, trainig):
        raise NotImplementedError()
    
class LuongAttention(BaseAttention):
    default_conf = {"local_scale":5, "attention_size":300}
    def __init__(self, conf=None):
        super().__init__(conf)
        self.local_scale = self.conf["local_scale"]
        self.attention_size = self.conf["attention_size"]
        self.middle_hidden = int(0.5 * self.attention_size)
        model_params = [{"name":"dense", "num_hidden":self.middle_hidden, "activation":tf.tanh}]
        self.local_ff = FeedForward(model_params, scope_name="local_ff")
        model_params = [{"name":"dense", "num_hidden":self.attention_size, "activation":tf.tanh}]
        self.attention_ff = FeedForward(model_params, scope_name="attention_ff")   
    
    def _get_global_attentions(self, query, source_outputs, source_inputs, training):
        """Calulate global alignments
        
        Args:
            query: tensor with shape=(batch, output_size), decoder RNN output
            source_outputs: tensor with shape=(batch, time_length, output_size), 
                entire outputs of encoder RNN
            source_inputs: tensor with shape=(batch, time_length)
            training: tensor bool
            
        Returns:
            alignments with shape = (batch, time_length)
        """
        query = tf.expand_dims(query, 1)
        scale = get_variable("global_scale", [],
                                     dtype=tf.float32,
                                     initializer=tf.ones_initializer)
        scale = scale / tf.constant(np.sqrt(self.attention_size))
        _score = tf.matmul(query, source_outputs, transpose_b=True)
        _score = tf.squeeze(_score, [1]) #shape=(batch, output_size)
        _alignments = tf.nn.softmax(_score)
        # We will not put attention to padded input
        weights = tf.cast(source_inputs >= 1, tf.float32)
        _alignments = weights * _alignments
        length = tf.shape(_alignments)[1]
        alignments = _alignments / tf.tile(tf.reduce_sum(_alignments, 1, keep_dims=True), [1, length])
        return alignments
    
    def _get_local_attentions(self, query, source_outputs, source_inputs, training):
        """Calulate local alignments
        
        Args:
            query: tensor with shape=(batch, output_size), decoder RNN output
            source_outputs: tensor with shape=(batch, time_length, output_size), 
                entire outputs of encoder RNN
            source_inputs: tensor with shape=(batch, time_length)
            training: tensor bool
            
        Returns:
            alignments with shape = (batch, time_length)
        """
        batch_size, length, output_size = tf.unstack(tf.shape(source_outputs))
        scale = get_variable("local_scale", [],
                             dtype=tf.float32,
                             initializer=tf.ones_initializer)
        scale = scale / tf.constant(np.sqrt(self.middle_hidden))
        u = self.local_ff(query, training) / scale
        # Consider only non zero index range, the size will be (batch, 1)
        S = tf.cast(tf.reduce_sum(tf.cast(source_inputs >= 1, tf.float32), axis=1, keep_dims=True), tf.float32)
        v_local = tf.get_variable("v_local", [self.middle_hidden], dtype=tf.float32)
        v_local = tf.expand_dims(v_local, [1])
        # Size will be [batch_size, 1]
        pt = S * tf.nn.sigmoid(tf.matmul(u, v_local))
        pt = tf.tile(pt, [1, length])
        # pt has shap=(batch, length) and values are the same along time axis
        sig = tf.constant(0.5) * self.local_scale
        # Index value will be in [1, S]
        index = tf.cast(tf.range(tf.constant(1), length + tf.constant(1)), tf.float32)
        index = tf.expand_dims(index, [0])
        index = tf.tile(index, [batch_size, 1])
        # Size of pt and  index has to be [batch_size, length]
        alignments = tf.exp(-0.5 * (pt - index)**2 / sig**2)
        return alignments
    
    def _get_attentions(self, query, source_outputs, source_inputs, training):
        local_alignments = self._get_local_attentions(query, source_outputs, source_inputs, training)
        global_alignments = self._get_global_attentions(query, source_outputs, source_inputs, training)
        alignments = global_alignments * local_alignments
        _alignments = tf.expand_dims(alignments, 1)
        context = tf.matmul(_alignments, source_outputs)
        # context shape is (batch_size, 1, output_size)
        context = tf.squeeze(context, [1])
        pre_attentions = tf.concat((context, query), axis=1)
        attentions = self.attention_ff(pre_attentions, training)
        return attentions, alignments

In [97]:
import tensorflow as tf
import tensorflow.contrib.rnn as rnn
import tensorflow.contrib.seq2seq as seq2seq
from hedgeable_ai.models.nn import BaseModel, get_shape, get_length
import tensorflow.contrib.rnn as rnn

import tensorflow as tf

from hedgeable_ai.models.nn.params import nn_is_logit
from hedgeable_ai.models.nn import BaseNN, get_shape

from hedgeable_ai.models.nn.rnn import get_cell

count = 0
class DialogueAgent(BaseNN):
    def __init__(self, processor, emb_size=300, maxlen=20,
                 conf=None, additional_length=3, clip_val=5.0, 
                 is_attention=False, *args, **kwargs):
        self.emb_size = emb_size
        self.clip_val = clip_val
        # leave index 0 for padding and 1 for  <eos>
        self.vocab_size = processor.vocab_size + 2
        self.maxlen = maxlen
        self.additional_length = additional_length
        self.is_attention = is_attention
        super().__init__(processor=processor, conf=conf, *args, **kwargs)
        
    def _build_graph(self):
        """Build tensorflow graph
        
        Note:
            You build graphs for output and input, which will be used 
            for training and prediction.
        """
        # Build Basic Netwoiork
        self.attention_model = LuongAttention(self.conf)
        W_output = tf.Variable(
            tf.random_uniform([self.attention_model.attention_size, self.vocab_size], -0.1, 0.1), dtype=tf.float32)
        b_output = tf.Variable(tf.zeros([self.vocab_size]), dtype=tf.float32)
        
        self.encoder_input = tf.placeholder(tf.int32, shape=(None, None), name="encoder_input")
        self.decoder_target = tf.placeholder(tf.int32, shape=(None, None), name="decoder_target")
        self.training = tf.placeholder(tf.bool, (), name="trainig")
        encoder_length = get_length(self.encoder_input)
        decoder_length = get_length(self.decoder_target)
        # Encoder
        encoder_cell = get_cell(self.conf["model"])
        embeddings = tf.Variable(
            tf.random_uniform([self.vocab_size, self.emb_size], -0.1, 0.1, dtype=tf.float32))
        encoder_input_embedded = tf.nn.embedding_lookup(embeddings, self.encoder_input)
        encoder_outputs, encoder_final_state = tf.nn.dynamic_rnn(
            encoder_cell, encoder_input_embedded, sequence_length=encoder_length, 
            dtype=tf.float32, time_major=False, scope="encoder")

        # Decoder
        decoder_cell = get_cell(self.conf["model"])
        batch_size, encoder_max_time = tf.unstack(tf.shape(self.encoder_input))
        decoder_output_size = self._get_output_size(self.conf["model"])
        # We use a decoder length 10 words longer than encoder
        max_decoder_length = encoder_length + tf.constant(self.additional_length)
        self.max_decoder_length = max_decoder_length
        # Prepare for padding and EOS
        eos_time_slice = tf.ones([batch_size], dtype=tf.int32, name='EOS')
        eos_step_embedded = tf.nn.embedding_lookup(embeddings, eos_time_slice)
        pad_time_slice = tf.ones([batch_size], dtype=tf.int32, name='PAD')
        pad_step_embedded = tf.zeros_like(eos_step_embedded)
        
        def loop_fn_initial():
            initial_elements_finished = (0 >= max_decoder_length)
            initial_input = eos_step_embedded
            initial_cell_state = encoder_final_state
            initial_cell_output  = None
            initial_loop_state = None
            return (initial_elements_finished,
                initial_input,
                initial_cell_state,
                initial_cell_output,
                initial_loop_state)
        
        def loop_fn_transition(time, previous_output, previous_state, previous_loop_state):
            if self.is_attention:
                previous_output, alignment = self.attention_model(
                    previous_output, encoder_outputs, self.encoder_input, self.training)
            output_logits = tf.matmul(previous_output, W_output) + b_output
            prediction  = tf.argmax(output_logits, axis=1)
            def get_next_input():
                next_input = tf.nn.embedding_lookup(embeddings,  prediction)
                return next_input
            elements_finished = (time >= max_decoder_length)
            finished = tf.reduce_all(elements_finished)
            next_input = tf.cond(finished, lambda: pad_step_embedded, get_next_input)
            state  = previous_state
            loop_state = previous_loop_state
            output = previous_output
            return (elements_finished, 
                next_input,
                state,
                output,
                loop_state)
        
        def loop_fn(time, previous_output, previous_state,  previous_loop_state):
            if previous_state is None:
                assert previous_output is None and previous_state is None
                return loop_fn_initial()
            else:
                return loop_fn_transition(time, previous_output, previous_state,  previous_loop_state)
        
        decoder_outputs_ta, decoder_final_state, _ = tf.nn.raw_rnn(decoder_cell, loop_fn)
        decoder_outputs = decoder_outputs_ta.stack()
        decoder_max_steps, decoder_batch_size, decoder_outputs_size = tf.unstack(tf.shape(decoder_outputs))
        self.max_steps = decoder_max_steps
        self.decoder_batch_size = decoder_batch_size
        decoder_outputs_size = self.attention_model.attention_size
        self.decoder_outputs = decoder_outputs
        decoder_outputs_flat = tf.reshape(decoder_outputs, (-1, decoder_outputs_size))
        decoder_logits_flat = tf.matmul(decoder_outputs_flat, W_output) + b_output
        decoder_logits = tf.reshape(decoder_logits_flat, (decoder_max_steps, decoder_batch_size, self.vocab_size))
        decoder_logits = tf.transpose(decoder_logits, [1, 0, 2])
        self.decoder_prediction = tf.argmax(decoder_logits, 2)
        self.decoder_logits = decoder_logits
        
        # Optimization
        cross_entropy = tf.nn.softmax_cross_entropy_with_logits(
            labels=tf.one_hot(self.decoder_target, depth=self.vocab_size, dtype=tf.float32),
            logits=decoder_logits)
        # cross_entropy = (tf.nn.softmax(self.decoder_logits) - tf.one_hot(self.decoder_target, depth=self.vocab_size, dtype=tf.float32))**2
        # We do not optimize for negative labels
        # weights = tf.cast(self.decoder_target >= 0, tf.float32)
        # self.weights = weights
        # self.loss = tf.reduce_mean(cross_entropy * weights)
        self.loss = tf.reduce_mean(cross_entropy)
        
        self.learning_rate_op = self._get_learning_rate()
        update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        with tf.control_dependencies(update_ops):
            self.optimizer = self._get_optimizer(self.optimizer_name, self.learning_rate_op, self.optimizer_conf)
            grads_vars = self.optimizer.compute_gradients(self.loss)
            clipped_grads_vars = [
                (tf.clip_by_norm(gv[0], clip_norm=self.clip_val), gv[1]) 
                for gv in grads_vars]
            self.train_step = self.optimizer.apply_gradients(clipped_grads_vars)
        
    def _optimize(self, batch_X, batch_y, *args, **kwargs):
        global count
        batch_X = [x[::-1] for x in batch_X[0]]
        batch_X, Xlen = self.processor.batch_padding(batch_X, self.maxlen)
        length = np.max(Xlen) + self.additional_length
        batch_y = self._batch_padding(batch_y, length)
        feed_dict = {self.encoder_input: batch_X,
                     self.decoder_target: batch_y,
                     self.training: False}
        
        # print(np.argmax(logits, 2))
        # print("batch_y", batch_y)
        # print(decoder_batch)
        # print(one_hot)
        if count % 1000 == 0:
            logits = self.sess.run(self.decoder_logits, feed_dict=feed_dict)
            # alignments = self.sess.run(self.alignments, feed_dict=feed_dict)
            decoder_batch = self.sess.run(self.decoder_batch_size, feed_dict=feed_dict)
            print("predition", np.argmax(logits, 2))
            print("max", np.max(np.argmax(logits, 2)))
        # print("alignments", logits)
        # print("logits", logits)
        # print(np.max(np.argmax(logits, 2)))
        count += 1
        _, loss = self.sess.run([self.train_step, self.loss], feed_dict=feed_dict)
        return loss
    
    def _get_output_size(self, conf):
        if isinstance(conf, list) or isinstance(conf, tuple):
            x = conf[-1]
        else:
            x = conf
        return x["num_units"]
    
    def generate_sentences(self, sentences):
        X = [self.processor.encode(sentence) for sentence in sentences]
        # X = sentences
        X = [x_[::-1] for x_ in X]
        X, Xlen = self.processor.batch_padding(X, self.maxlen)
        feed_dict = {self.encoder_input: X,
                     self.training: True}
        word_idx = self.sess.run(self.decoder_prediction, feed_dict=feed_dict)
        logits =  self.sess.run(self.decoder_logits, feed_dict=feed_dict)
        print(word_idx)
        return [self.processor.decode(i) for i in word_idx], logits
    
    def _batch_padding(self, batch, length):
        EOS = 1
        PAD = 0
        padded_batch = []
        for x in batch:
            x = list(x)
            if len(x) > length:
                x = x[:length]
            elif len(x) < length:
                x.append(EOS)
            while len(x) < length:
                x.append(PAD)
            padded_batch.append(x)
        return np.array(padded_batch)

In [99]:
import tensorflow as tf
import numpy as np


conf = {
        "learning_rate": .5,
        "learning_rate_minimum": 1e-3,
        "learning_rate_decay": 0.99,
        "learning_rate_decay_step": 1,
        "batch_size": 128,
        "model_dir": "./logs",
        "load_file_path": None,
        "save_file_path": None,
        "log_freq": 1,
        "optimizer":"gd",
        "model":[{"name":"lstm", "num_units":512},
                {"name":"lstm", "num_units":512},],
        # "model":[{"name":"lstm", "num_units":500},
        #         {"name":"lstm", "num_units":500}],
        "attention_size": 512
}

tf.reset_default_graph()
agent = DialogueAgent(processor, maxlen=10, conf=conf, additional_length=3, is_attention=False)

train_X = processor.data[:-1]
train_y = processor.data[1:]
agent.fit(train_X, train_y, num_epochs=10000, batch_bar=False, log_freq=1, batch_log_freq=100)

/home/tomoaki/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "




  0%|          | 0/10000 [00:00<?, ?it/s]

Model saved in file: params/model.ckpt






  0%|          | 1/10000 [01:01<170:37:50, 61.43s/it]

predition [[10738    36     1 ...,     0     0     0]
 [17914 17914     1 ...,     0     0     0]
 [10738    36     1 ...,     0     0     0]
 ..., 
 [    0     1     1 ...,     0     0     0]
 [10738    36     1 ...,     0     0     0]
 [10738 17914     1 ...,     0     0     0]]
max 17914






  0%|          | 2/10000 [02:03<170:50:01, 61.51s/it]

...

predition [[ 7949 17914    36 ...,    36     1     0]
 [ 7949     1     0 ...,     0     0     0]
 [ 7949  7949 10552 ...,     0     0     0]
 ..., 
 [ 7949  8492  8492 ...,     0     0     0]
 [10738 17914     0 ...,    36    36    36]
 [ 7949  8492 17914 ...,     0     0     0]]
max 17914






 13%|█▎        | 1337/10000 [23:03:44<148:43:29, 61.80s/it]



 13%|█▎        | 1338/10000 [23:04:45<148:37:46, 61.77s/it]

predition [[ 7949     1     0 ...,     0     0     0]
 [ 7949  8492 10552 ...,    36    36     1]
 [ 7949  8492     0 ...,     0     0     0]
 ..., 
 [10472    36     1 ...,     0     0     0]
 [ 7949    36    36 ...,     0     0     0]
 [10738  8492     1 ...,     0     0     0]]
max 17914






 13%|█▎        | 1339/10000 [23:05:47<148:39:23, 61.79s/it]



 13%|█▎        | 1340/10000 [23:06:49<148:34:48, 61.77s/it]

predition [[17496    36    36 ...,     0     0     0]
 [ 7949  8492  8492 ...,     0     0     0]
 [ 7949  7949 10552 ...,    36    36     1]
 ..., 
 [ 7949     1 17914 ...,     0     0     0]
 [ 7949  8492 10552 ...,    36     0     0]
 [ 7949 17914 17914 ...,     0     0     0]]
max 17914
Model saved in file: params/model.ckpt


In [100]:
prediction, logits = agent.generate_sentences(texts[:1000])

[[17496     1     0 ...,     0     0     0]
 [ 7949  8492     1 ...,     0     0     0]
 [10738    36     1 ...,     0     0     0]
 ..., 
 [ 7949  8492  8492 ...,     0     0     0]
 [ 7949  8492 10552 ...,     1     0     0]
 [17914 17914    36 ..., 15868    36  7949]]


In [101]:
print(prediction)

[['what'], ['i', 'is'], ['oh', '.'], ['i', 'you', 'you'], ['.'], ['i', 'is', 'is'], ['i', 'is', 'not', 'is', '.', 'i', 'is', 'a', 'is', '.'], ['i', 'is', 'not', 'not', '.', 'is', '.', '.', '.'], ['i', 'i', 'i', '.', '.', '.', '.', '.', '.', '.', '.'], ['i', 'is', '.', '.', '.', '.'], ['i', 'is', 'not', '.', '.', '.', 'you', 'you', 'the', '.', 'you', 'i', '.'], ['what'], ['i'], ['i', '.'], ['i', 'is'], ['oh', 'is'], ['oh', '.', '.', 'is'], ['yeah'], ['i', 'is', '.', '.'], ['oh', 'is', '.', '.'], ['i'], ['you', 'is', '.'], ['hey'], ['oh', 'is'], ['i', 'is'], ['oh', 'is'], ['i', 'i'], ['oh', 'is', 'is'], ['i', '.', '.'], ['oh', 'is', 'is'], ['oh', 'is', 'is'], ['i', 'is'], ['you', '.'], ['what'], ['i', 'is'], ['yeah', 'i', 'i', 'you', 'i', 'i', 'i', 'i', '.', 'you', 'you', '.', '.'], ['.'], ['what', 'is', '.', '.'], ['oh', 'is', 'is'], ['oh', 'is', 'is', 'is', '.'], ['oh', '.', '.', '.', '.', '.'], ['oh', '.'], ['i', '.', '.', '.', '.'], ['oh', 'is', 'is', 'is', 'is', 'is', 'is', '.', '.'

In [75]:
processor.vocab_size

17960

In [89]:
texts[0]

'You guys, you guys! Chef is going away. \n'